In [1]:
# head module import
from __future__ import print_function

from mbientlab.metawear import MetaWear, libmetawear, parse_value, create_voidp, create_voidp_int

from mbientlab.metawear.cbindings import *

from mbientlab.warble import * 
from threading import Event
from time import sleep

import csv
import time
import platform
import six # six is lib to coordinate py2 and py3
import sys

In [2]:
# check python version
if sys.version_info[0] == 2:

    range = xrange

In [9]:
# device scan and connect
selection = -1

devices = None # all available devices

scan_time = 10.0

while selection == -1:
    
    print("scanning for devices...")

    devices = {}
    # def scanner's result handler, add result to dvices
    def handler(result):
        devices[result.mac] = result.name
    
    BleScanner.set_handler(handler)
    BleScanner.start()
    # set scan time, waiting to scan;
    sleep(scan_time)
    BleScanner.stop()
    
    #select which to connect
    devices_index = 0
    for address, name in six.iteritems(devices):
        print("[%d] %s (%s)" % (devices_index, address, name))
        devices_index += 1
    msg = "Select your device (-1 to rescan): "
    selection = int(raw_input(msg) if platform.python_version_tuple()[0] == '2'
                    else input(msg))


scanning for devices...
[0] F1:3E:CC:8F:98:89 (MetaWear)
[1] F9:C7:50:6F:77:60 ()
[2] 41:0A:F9:64:A8:1D ()
Select your device (-1 to rescan): 0


In [10]:
# get chosed devices mac address                    
address = list(devices)[selection]
                    
print("Connecting to %s..." % (address))
# connect to device                   
device = MetaWear(address)
device.connect()
print("Connected")

print("Device information: " + str(device.info))
         

Connecting to F1:3E:CC:8F:98:89...
Connected
Device information: {'model': '5', 'hardware': '0.3', 'manufacturer': 'MbientLab Inc', 'serial': '049954', 'firmware': '1.4.2'}


In [8]:
libmetawear.mbl_mw_debug_reset(device.board)

In [11]:
# acc configuration
def acc_configuaration(device,acc_freq, acc_rng):
    acc_frequency = acc_freq
    acc_range = acc_rng
    libmetawear.mbl_mw_acc_set_odr(device.board, acc_frequency)
    libmetawear.mbl_mw_acc_set_odr(device.board, acc_range)
    libmetawear.mbl_mw_acc_write_acceleration_config(device.board);
    print("set acc frequency [%d], acc range [%d]" %(acc_freq, acc_rng))

In [12]:
# init signal and logger
def init_acc_signal_logger(device):
    # signal
    signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)
    # logger
    logger = create_voidp(lambda fn:libmetawear.mbl_mw_datasignal_log(signal, None, fn), resource = "acc_logger")
    print("finished init acc_signal and logger ")
    return logger

In [13]:
# connection setting
def init_connection_setting(device):
    libmetawear.mbl_mw_settings_set_connection_parameters(device.board, 7.5, 7.5, 0, 6000)
    sleep(1.0)
    print("finished connection setting")

In [14]:
# start log and acc sampling
def sampling_and_log_acc(device,time):
    libmetawear.mbl_mw_logging_start(device.board,0)
    libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
    libmetawear.mbl_mw_acc_start(device.board)
    print("start sampling data")
    # wait to log
    logging_time = time
    sleep(logging_time)
    # stop logging
    print("stop sampling data")
    libmetawear.mbl_mw_acc_stop(device.board)
    libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
    libmetawear.mbl_mw_logging_stop(device.board)
    

In [9]:
# downloading data setting
def init_download_data_setting():
    print("Setting Download data")
    # progress_update_handler
    global e
    e = Event()
    def progress_update_handler(context, entries_left, total_entries):
        if (entries_left == 0):
            e.set()
            print("Finished download data")
            
    fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler) # wrap function    
    download_handler = LogDownloadHandler(context = None, \
                                     received_progress_update = fn_wrapper, \
                                     received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), \
                                     received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))
    # data callback function
    def data_callback(context, p):
        print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p)))
        # print(cast(p.contents.value, POINTER(CartesianFloat)).contents.x) debug全靠输出
        timestamp = str(context)
        csvfile = open('acc_data_%s.csv'%timestamp, 'a',newline='')
        writer = csv.writer(csvfile)
        # writer.writerow(str(parse_value(p)))
        data = parse_value(p)
        writer.writerow([data.x, data.y,data.z])
        # writer.writerow([p.contents.value])
              
    # callback = FnVoid_VoidP_DataP(lambda ctx, p: print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p))))
    callback = FnVoid_VoidP_DataP(data_callback)
    print("finished init download setting ")
    return [download_handler, callback]

In [15]:
# downloading data setting
def init_download_data_setting():
    print("Setting Download data")
    # progress_update_handler
    global e
    e = Event()
    def progress_update_handler(context, entries_left, total_entries):
        if (entries_left == 0):
            e.set()
            print("Finished download data")
            
    fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler) # wrap function    
    download_handler = LogDownloadHandler(context = None, \
                                     received_progress_update = fn_wrapper, \
                                     received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), \
                                     received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))
    # data callback function
    def data_callback(context, p):
        print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p)))
        data = parse_value(p)
        filename = cast(context,c_wchar_p)
        csvfile = open(filename.value, 'a',newline='')
        writer = csv.writer(csvfile)
        writer.writerow([data.x, data.y,data.z])
      
    # callback = FnVoid_VoidP_DataP(lambda ctx, p: print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p))))
    callback = FnVoid_VoidP_DataP(data_callback)
    print("finished init download setting ")
    return [download_handler, callback]

In [16]:
# downloading data
def downloading_data(device,logger,callback,download_handler):
    print("Downloading data")
    timestamp = time.strftime("%y_%m_%d_%H_%M_%S")
    print(type(timestamp))
    print(timestamp)
    filename_ = ('acc_data_%s.csv'%timestamp)
    csvfile = open(filename_, 'w',newline='')
    filename = c_wchar_p(filename_)
    libmetawear.mbl_mw_logger_subscribe(logger, filename, callback)
    # libmetawear.mbl_mw_logger_subscribe(logger, filename, callback)
    libmetawear.mbl_mw_logging_download(device.board, 0, byref(download_handler))
    
    e.wait()

In [15]:
# downloading data
def downloading_data(device,logger,callback,download_handler):
    print("Downloading data")
    timestamp = c_int(int(time.time()))
    # csvfile = open('acc_data_%s.csv'%timestamp, 'w',newline='')
    # writer = csv.writer(csvfile)
    timestamp_ptr = pointer(timestamp)
    libmetawear.mbl_mw_logger_subscribe(logger, timestamp_ptr, callback)
    libmetawear.mbl_mw_logging_download(device.board, 0, byref(download_handler))
    
    e.wait()

In [17]:
logger = None
callback = None
download_handler = None
e = None
acc_configuaration(device,25.0,4.0)
init_connection_setting(device)
logger = init_acc_signal_logger(device)


set acc frequency [25], acc range [4]
finished connection setting
finished init acc_signal and logger 


In [18]:
download_setting = init_download_data_setting()
download_handler = download_setting[0]
callback = download_setting[1]
sampling_and_log_acc(device,5.0)
downloading_data(device,logger,callback,download_handler)


Setting Download data
finished init download setting 
start sampling data
stop sampling data
<class 'str'>
19_03_13_20_54_42
{epoch: 1552481416177, value: {x : -0.076, y : -0.012, z : -1.005}}
{epoch: 1552481416501, value: {x : -0.071, y : -0.012, z : -1.009}}
{epoch: 1552481416826, value: {x : -0.073, y : -0.010, z : -1.014}}
{epoch: 1552481417150, value: {x : -0.070, y : -0.018, z : -0.995}}
{epoch: 1552481417473, value: {x : -0.065, y : -0.009, z : -1.005}}
{epoch: 1552481417799, value: {x : -0.073, y : -0.014, z : -1.005}}
{epoch: 1552481418122, value: {x : -0.073, y : -0.015, z : -1.009}}
{epoch: 1552481418446, value: {x : -0.070, y : -0.022, z : -1.011}}
{epoch: 1552481418771, value: {x : -0.070, y : -0.012, z : -1.016}}
{epoch: 1552481419095, value: {x : -0.075, y : -0.012, z : -1.004}}
{epoch: 1552481419419, value: {x : -0.078, y : -0.023, z : -1.006}}
{epoch: 1552481419744, value: {x : 0.209, y : -0.404, z : -1.323}}
{epoch: 1552481420068, value: {x : -0.088, y : 0.103, z : -0

In [17]:
sampling_and_log_acc(device,5.0)
downloading_data(device,logger,callback,download_handler)

start sampling data
stop sampling data
Finished download data
{epoch: 1552448207605, value: {x : -0.039, y : -0.013, z : 1.020}}
{epoch: 1552448207929, value: {x : -0.039, y : -0.012, z : 1.033}}
{epoch: 1552448208252, value: {x : -0.032, y : 0.000, z : 1.039}}
{epoch: 1552448208578, value: {x : -0.020, y : 0.107, z : 1.089}}
{epoch: 1552448208901, value: {x : -0.036, y : 0.024, z : 1.015}}
{epoch: 1552448209225, value: {x : -0.031, y : 0.033, z : 1.019}}
{epoch: 1552448209549, value: {x : -0.031, y : 0.021, z : 1.017}}
{epoch: 1552448209874, value: {x : -0.037, y : 0.003, z : 1.020}}
{epoch: 1552448210198, value: {x : -0.025, y : -0.001, z : 1.024}}
{epoch: 1552448210521, value: {x : -0.034, y : -0.010, z : 1.031}}
{epoch: 1552448210845, value: {x : -0.033, y : -0.014, z : 1.019}}
{epoch: 1552448211169, value: {x : -0.037, y : 0.011, z : 1.033}}
{epoch: 1552448211494, value: {x : -0.030, y : 0.017, z : 1.018}}
{epoch: 1552448211818, value: {x : -0.042, y : 0.029, z : 1.004}}
{epoch: 1

In [8]:
# init signal and logger
# signal
signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)
# logger
logger = create_voidp(lambda fn:libmetawear.mbl_mw_datasignal_log(signal, None, fn), resource = "acc_logger")

In [ ]:
# acc configuration
acc_frequency = 25.0
acc_range = 4.0
libmetawear.mbl_mw_acc_set_odr(device.board, acc_frequency)
libmetawear.mbl_mw_acc_set_odr(device.board, acc_range)
libmetawear.mbl_mw_acc_write_acceleration_config(device.board);

In [ ]:
# connection setting
libmetawear.mbl_mw_settings_set_connection_parameters(device.board, 7.5, 7.5, 0, 6000)
sleep(1.0)

In [ ]:
# downloading data setting
print("Downloading data")
# progress_update_handler
e = Event()
def progress_update_handler(context, entries_left, total_entries):
    if (entries_left == 0):
        e.set()
            
fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler) # wrap function    
download_handler = LogDownloadHandler(context = None, \
                                     received_progress_update = fn_wrapper, \
                                     received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), \
                                     received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))
# data callback function
callback = FnVoid_VoidP_DataP(lambda ctx, p: print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p))))


In [ ]:
# start log and acc sampling
libmetawear.mbl_mw_logging_start(device.board,0)
libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
libmetawear.mbl_mw_acc_start(device.board)
print("start sampling data")
# wait to log
logging_time = 2.0
sleep(logging_time)
# stop logging
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
libmetawear.mbl_mw_logging_stop(device.board)

In [ ]:
# downloading data
libmetawear.mbl_mw_logger_subscribe(logger, None, callback)
libmetawear.mbl_mw_logging_download(device.board, 0, byref(download_handler))
e.wait()

In [19]:
# device info and status check
print("Device information: " + str(device.info))
print("Device connet to: " + str(device.connect))
print("Device isconnet: " + str(device.is_connected))
print("Device ondisconnet: " + str(device.on_disconnect))

Device information: {'model': '5', 'hardware': '0.3', 'manufacturer': 'MbientLab Inc', 'serial': '049954', 'firmware': '1.4.2'}
Device connet to: <bound method MetaWear.connect of <mbientlab.metawear.metawear.MetaWear object at 0x0000017D3F1D35F8>>
Device isconnet: True
Device ondisconnet: None


In [ ]:
try:
    # acc configuration
    acc_frequency = 25.0
    acc_range = 4.0
    libmetawear.mbl_mw_acc_set_odr(device.board, acc_frequency)
    libmetawear.mbl_mw_acc_set_odr(device.board, acc_range)
    libmetawear.mbl_mw_acc_write_acceleration_config(device.board);
    # init signal and logger
    # signal
    signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)
    # logger
    logger = create_voidp(lambda fn:libmetawear.mbl_mw_datasignal_log(signal, None, fn), resource = "acc_logger")
    
    # start log and acc sampling
    libmetawear.mbl_mw_logging_start(device.board,0)
    libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
    libmetawear.mbl_mw_acc_start(device.board)
    print("start sampling data")
    # wait to log
    logging_time = 2.0
    sleep(logging_time)
    
    # stop logging
    libmetawear.mbl_mw_acc_stop(device.board)
    libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
    libmetawear.mbl_mw_logging_stop(device.board)
    
    # downloading data
    print("Downloading data")
    libmetawear.mbl_mw_settings_set_connection_parameters(device.board, 7.5, 7.5, 0, 6000)
    sleep(1.0)
    
    # download handler
    # progress_update_handler
    e = Event()
    def progress_update_handler(context, entries_left, total_entries):
        if (entries_left == 0):
            e.set()
            
    fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler) # wrap function
    
    download_handler = LogDownloadHandler(context = None, \
                                         received_progress_update = fn_wrapper, \
                                         received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), \
                                         received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))
    callback = FnVoid_VoidP_DataP(lambda ctx, p: print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p))))
    libmetawear.mbl_mw_logger_subscribe(logger, None, callback)
    libmetawear.mbl_mw_logging_download(device.board, 0, byref(download_handler))
    e.wait()
except RuntimeError as err:

    print(err)

finally:

    print("Resetting device")

    e = Event()

    device.on_disconnect = lambda status: e.set()
    # reset board setting and disconnect
    libmetawear.mbl_mw_debug_reset(device.board)

    e.wait()

In [ ]:
# get chosed devices mac address                    
address = list(devices)[selection]
                    
print("Connecting to %s..." % (address))
# connect to device                   
device = MetaWear(address)
device.connect()
print("Connected")

print("Device information: " + str(device.info))
         

In [ ]:
# signal
signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)
# logger
logger = create_voidp(lambda fn:libmetawear.mbl_mw_datasignal_log(signal, None, fn), resource = "acc_logger")
libmetawear.mbl_mw_logging_start(device.board,0)
    
# start acc sampling
libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
libmetawear.mbl_mw_acc_start(device.board)
print("start sampling data")
 # wait to log
logging_time = 2.0
sleep(logging_time)
    
# stop logging
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
libmetawear.mbl_mw_logging_stop(device.board)
    
# downloading data
print("Downloading data")
libmetawear.mbl_mw_settings_set_connection_parameters(device.board, 7.5, 7.5, 0, 6000)
sleep(1.0)
e = Event()
def progress_update_handler(context, entries_left, total_entries):
    if (entries_left == 0):
        e.set()
            
fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler) # wrap function
download_handler = LogDownloadHandler(context = None, \
                                    received_progress_update = fn_wrapper, \
                                    received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), \
                                    received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))
callback = FnVoid_VoidP_DataP(lambda ctx, p: print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p))))
libmetawear.mbl_mw_logger_subscribe(logger, None, callback)
libmetawear.mbl_mw_logging_download(device.board, 0, byref(download_handler))
e.wait()

In [ ]:
# start acc sampling
libmetawear.mbl_mw_logging_start(device.board,0)
libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
libmetawear.mbl_mw_acc_start(device.board)
print("start sampling data")
 # wait to log
logging_time = 1.0
sleep(logging_time)
    
# stop logging
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
libmetawear.mbl_mw_logging_stop(device.board)
    
# downloading data
print("Downloading data")
libmetawear.mbl_mw_settings_set_connection_parameters(device.board, 7.5, 7.5, 0, 6000)
sleep(1.0)
e = Event()
libmetawear.mbl_mw_logger_subscribe(logger, None, callback)
libmetawear.mbl_mw_logging_download(device.board, 0, byref(download_handler))
e.wait()

In [ ]:
device.disconnect()
sleep(1.0)
print("Disconnected")         